In [73]:
import pandas as pd
import json
from natsort import natsorted

In [3]:
genera = pd.read_csv('final_genera_75.tsv', delimiter='\t')
parsed_glottolog = pd.read_csv('languages_with_pages.tsv', sep='\t', index_col=0)

In [14]:
genera = pd.merge(genera, parsed_glottolog)[['glottocode', 'genus', 'name', 'pages', 'macroareas']]

In [16]:
macroareas = {}

In [28]:
for macro in genera.macroareas.unique():
    if pd.isna(macro):
        continue
    df = genera[genera.macroareas == macro]
    n_genera = len(df.genus.unique()) + sum(df.genus == 'isolate') - 1
    macroareas[macro.strip()] = n_genera

In [32]:
macroareas_percentage = {
    x: y / sum(macroareas.values()) for x, y in macroareas.items()
}

In [88]:
def convert_pages(pages):
    pages = eval(pages)
    return [x for x in pages if not isinstance(x, str)]

In [122]:
n_langs = 50
given_genera = ['Tungusic', 'Saami', 'Northwest Caucasian', 'Slavic', 'Marind-Yaqay']

n_by_macroarea = {
    x: round(y * n_langs) for x, y in macroareas_percentage.items()
}

sample_by_macroarea = {}
added_genera = []

for macro in n_by_macroarea:
    n = n_by_macroarea[macro]
    df = genera[genera.macroareas == '\n' + macro].copy()
    df.pages = df.pages.apply(lambda x: sorted(convert_pages(x), reverse=True))
    sorted_genera = df.sort_values('pages', ascending=False).genus.dropna().unique().tolist()
    needed_genera = []
    for gen in given_genera:
        if gen in sorted_genera:
            sorted_genera.remove(gen)
            needed_genera.append(gen)
            n -= 1
    for gen in added_genera:
        if gen in sorted_genera:
            sorted_genera.remove(gen)
    
    sorted_genera.remove('Creoles and Pidgins')
    sorted_genera.remove('Sign Languages')

    needed_genera.extend(sorted_genera[:n_by_macroarea['Eurasia']])
    added_genera.extend(needed_genera)
    
    sample_by_macroarea[macro] = needed_genera

In [123]:
sample_by_macroarea

{'Papunesia': ['Marind-Yaqay',
  'Oceanic',
  'Eleman',
  'Western Plains Austronesian',
  'North Borneo',
  'Kalam-Kobon',
  'Central Luzon',
  'Greater Central Philippine'],
 'South America': ['Panoan',
  'Nadahup',
  'Japura-Colombia',
  'isolate',
  'Quechuan',
  'Bolivia-Parana',
  'Araucanian'],
 'Eurasia': ['Tungusic',
  'Saami',
  'Northwest Caucasian',
  'Slavic',
  'Romance',
  'Germanic',
  'Ugric',
  'Semitic',
  'Ainu',
  'Baltic',
  'Finnic'],
 'North America': ['Mayan',
  'Eskimo',
  'Athapaskan',
  'Haida',
  'Tlingit',
  'Eyak',
  'Tarahumaran'],
 'Africa': ['Peul-Serer',
  'Gbe',
  'Bantu',
  'Barito',
  'Berber',
  'Ta-Ne-Omotic',
  'Biu-Mandara'],
 'Australia': ['Western Pama-Nyungan',
  'Nyulnyulan',
  'Eastern Gunwinyguan',
  'Tangkic',
  'Northern Pama-Nyungan',
  'Marne',
  'Western Gunwinyguan']}